In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import tensorflow as tf
slim = tf.contrib.slim
tf.logging.set_verbosity(tf.logging.INFO)

import sys
import glob

IMAGENET_DATA = '/gpfs01/bethge/data/imagenet'
VGG_CKPT_FILE = '/gpfs01/bethge/data/tf-model-checkpoints/vgg_normalized_complete.ckpt'
SLIM_MODELS_PATH = '/gpfs01/bethge/home/aecker/lab/projects/tf-models/slim'

if SLIM_MODELS_PATH not in sys.path:
    sys.path.append(SLIM_MODELS_PATH)
from datasets import imagenet
from preprocessing import vgg_preprocessing

import vgg

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
with tf.Graph().as_default():
    
    # preprocessing
    with tf.device('/cpu:0'):
        dataset = imagenet.get_split('validation', IMAGENET_DATA)
        provider = slim.dataset_data_provider.DatasetDataProvider(
            dataset, num_readers=5, shuffle=False)
        raw_image, raw_label = provider.get(['image', 'label'])
        image = vgg_preprocessing.preprocess_image(
            raw_image, 224, 224, is_training=False, resize_side_min=256)
        label = tf.sub(raw_label, 1) # 1..1000 to 0..999
        
    # VGG
    images, labels = tf.train.batch([image, label], batch_size=32, capacity=256)
    end_points = vgg.loss_vgg(images, subtract_mean=False, final_endpoint='fc8')
    predictions = tf.squeeze(end_points['fc8'], [1, 2])
    
    # evaluation ops
    metric_op, update_op = slim.metrics.streaming_recall_at_k(predictions, labels, k=5)
    
    # initialization and checkpoint loading 
    initial_op = tf.group(
        tf.initialize_all_variables(),
        tf.initialize_local_variables())
    restorer = tf.train.Saver(slim.get_model_variables())
    
    # run evaluation session
    with tf.Session() as sess:
        sess.run(initial_op)
        restorer.restore(sess, VGG_CKPT_FILE)
        with slim.queues.QueueRunners(sess):
            metric_values = slim.evaluation.evaluation(
                sess,
                num_evals=10,
                initial_op=None,
                eval_op=update_op,
                final_op=metric_op)

# Output performance metric
print(metric_values)

INFO:tensorflow:Executing eval ops
INFO:tensorflow:Executing eval_op 1/10
INFO:tensorflow:Executing eval_op 2/10
INFO:tensorflow:Executing eval_op 3/10
INFO:tensorflow:Executing eval_op 4/10
INFO:tensorflow:Executing eval_op 5/10
INFO:tensorflow:Executing eval_op 6/10
INFO:tensorflow:Executing eval_op 7/10
INFO:tensorflow:Executing eval_op 8/10
INFO:tensorflow:Executing eval_op 9/10
INFO:tensorflow:Executing eval_op 10/10
INFO:tensorflow:Executing final op


0.9
